# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# All data is present in the ./event_data directory. 
# Create a list of all the files in that directory.
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# Read all the CSV files into memory.
full_data_rows_list = [] 
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)

        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Write the files out to a single file.
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


The generated file (event_datafile_new.csv) contains the following columns.

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the file.

<img src="images/image_event_datafile_new.jpg">

### Count the data in the csv

In [6]:
# Count the amount of rows in the file for sanity checking and debugging.
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Setting up the Cassandra Cluster

#### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Helper functions used in the remainder of the assignment.

In [10]:
def drop_table(table_name):
    """
    Drops the table of given name, if it exists.
    """
    query = "DROP TABLE IF EXISTS {};".format(table_name)
    try:
        session.execute(query)
    except Exception as e:
        print(e)


def run_create_query(query):
    """
    Runs a query without returning the result. 
    Used to create tables.
    """
    try:
        session.execute(query)
    except Exception as e:
        print(e)


def sanitizeValues(values):
    """
    This function sanitizes the inputs of the CSV files. 
    It converts all the numerical fields into the right type 
    by parsing them using int and float.
    """
    #[artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId]
    values[3] = int(values[3])
    values[5] = float(values[5])
    values[8] = int(values[8])
    values[10] = int(values[10])
    return values 


def insert_all_from_csv(session, query):
    """
    Given an insert query, inserts the values
    from the csv file into the database.
    Sanitizes values before inserting.
    """
    file = 'event_datafile_new.csv'

    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            try:
                session.execute(query, sanitizeValues(line))
            except Exception as e:
                print(e)


def run_select(query):
    """
    Runs the given query and prints the results.
    """
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)
        
    for row in rows:
        print(row)

## Create queries to ask the following three questions of the data

### Query 1 
> Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [11]:
# Create the table.
# The table is dropped first so that subsequent runs dont 
# insert duplicate values into the database.
drop_table("events_by_session_id_and_items")

# The primary key is chosen with the following reasoning. 
# A session contains multiple items, and within a session "itemInSession" 
# is unique, so those two values together are a unique identifier for a row.
# Because it makes sense to group sessions into a single partition the 
# primary key consists of two partition keys, instead of a single partition key and 
# a clustering key.
query = """CREATE TABLE IF NOT EXISTS events_by_session_id_and_items
           (artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text, location text, sessionId int, song text, userId int,
            PRIMARY KEY ((sessionId, itemInSession)))"""
            
run_create_query(query)

In [12]:
# Populate the table.
query = """ 
INSERT INTO events_by_session_id_and_items 
(artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

insert_all_from_csv(session, query)

KeyboardInterrupt: 

In [ ]:
# Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
query = """
        select artist, song, length from events_by_session_id_and_items where sessionId=338 and itemInSession=4;
        """

run_select(query)
                    

### Query 2 
> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [ ]:
# Create the table.
# The table is dropped first so that subsequent runs dont 
# insert duplicate values into the database.
drop_table("events_by_userid_and_sessionid")

# The primary key is chosen with the following reasoning. 
# A user id is unique, but does not uniquely identify a value. 
# The query wants us to filter on sessionid too. 
# These two values can not uniquely identify a row either. 
# When we add itemInSession as well, we have a unique row. 
# We put itemInSession as last clustering key because we don't want
#  to filter based on that, but we need it to order the values.
# The primary key consists of two composite keys because it makes sense
# to cluster the data together for a given user and given session
#  for this query in a single partition for performance reasons.
query = """CREATE TABLE IF NOT EXISTS events_by_userid_and_sessionid
           (artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text, location text, sessionId int, song text, userId int,
            PRIMARY KEY ((userId, sessionId), itemInSession))"""
run_create_query(query)

In [ ]:
# Populate the table.
query = """ 
INSERT INTO events_by_userid_and_sessionid 
(artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
insert_all_from_csv(session, query)

In [ ]:
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = "select artist, song, firstname, lastname from events_by_userid_and_sessionid where userId=10 and sessionId=182"
run_select(query)

### Query 3
> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
# Create the table.
# The table is dropped first so that subsequent runs dont 
# insert duplicate values into the database.
drop_table("events_by_song_and_sessionid")

# The primary key is chosen with the following reasoning. 
# A song does not uniquely identify a row, so we have to add additional data. 
# A song in a particular session is not unique, because it can be played multiple times. 
# A song, a session, and the item in that session however do identify a row.
# Because we are interested in all records for a given song it makes sense
# to cluster that data together. 
# We therefore only have a single partition key. 
# This should group all records for a given song together and increase performance of our query.
query = """CREATE TABLE IF NOT EXISTS events_by_song_and_sessionid
           (artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text, location text, sessionId int, song text, userId int,
            PRIMARY KEY (song, sessionId, itemInSession))"""
run_create_query(query)

In [ ]:
# Populate the table.
query = """ 
INSERT INTO events_by_song_and_sessionid 
(artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
insert_all_from_csv(session, query)

In [ ]:
# Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "select artist, song, firstname, lastname from events_by_song_and_sessionid where song='All Hands Against His Own';"
run_select(query)

### Drop the tables before closing out the sessions

In [ ]:
tables = ["events_by_session_id_and_items", "events_by_userid_and_sessionid", "events_by_song_and_sessionid"]
for table in tables: 
    drop_table(table)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()